In [1]:
import numpy as np
import sklearn
import pandas as pd
import joblib
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#import gensim


from sklearn.metrics import classification_report, f1_score, confusion_matrix,recall_score,precision_score,make_scorer
from sklearn.model_selection import StratifiedKFold, train_test_split, learning_curve,cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from statistics import mean, stdev
import lightgbm as lgb

In [2]:
tweets_label_test_df = pd.read_csv("clean_tweets_labelled_test.csv")
X_test,y_test=tweets_label_test_df["clean_data"],tweets_label_test_df["label"]

### ML models


#### SGD 

In [3]:
# Load the model from the file
SGD_from_joblib = joblib.load('ml/SGD_tfidf_vectorizer4.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'ml/SGD_tfidf_vectorizer4.pkl'

In [ ]:
# tfidf vectorizer transformation 
tfidf_obj = joblib.load('vect/_tfidf_vect_proxy_labelled4.pkl')
print(type(tfidf_obj))

X_test = tfidf_obj.transform(X_test.astype('U'))

In [ ]:

# Calculate the Score 
score = SGD_from_joblib.score(X_test, y_test)  
# Print the Score
print("Test Accuracy score: {0:.2f} %".format(100 * score))  

### Random Forest 

In [ ]:
# Load the model from the file
RF_from_joblib = joblib.load('ml/random_forest_tfidf_vectorizer4.pkl')

In [ ]:
# Calculate the Score 
score = RF_from_joblib.score(X_test, y_test)  
# Print the Score
print("Test Accuracy score: {0:.2f} %".format(100 * score))  

### SVM

In [ ]:
# Load the model from the file
SVM_from_joblib = joblib.load('ml/svm_tfidf_vectorizer4.pkl')

In [ ]:
# Calculate the Score 
score = SVM_from_joblib.score(X_test, y_test)  
# Print the Score
print("Test Accuracy score: {0:.2f} %".format(100 * score))  

### Naivye Bayes

In [ ]:
# Load the model from the file
NB_from_joblib = joblib.load('ml/naive_bayes_tfidf_vectorizer4.pkl')

In [ ]:
# Calculate the Score 
score = NB_from_joblib.score(X_test, y_test)  
# Print the Score
print("Test Accuracy score: {0:.2f} %".format(100 * score))  

### KNN

In [ ]:
# Load the model from the file
KNN_from_joblib = joblib.load('ml/KNeighborsClassifier_tfidf_vectorizer4.pkl')

In [ ]:
# Calculate the Score 
score = KNN_from_joblib.score(X_test, y_test)  
# Print the Score
print("Test Accuracy score: {0:.2f} %".format(100 * score))  

### Logistic Regression 

In [ ]:
# Load the model from the file
LR_from_joblib = joblib.load('ml/logistic_regression_tfidf_vectorizer4.pkl')

In [ ]:
# Calculate the Score 
score = LR_from_joblib.score(X_test, y_test)  
# Print the Score
print("Test Accuracy score: {0:.2f} %".format(100 * score))  

### Deep learning models

In [18]:
# Load the model from the file
#lstm_from_joblib = joblib.load('lstm_best_model.h5')

import gensim.downloader as api
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K
import tensorflow as tf
import keras
max_words = 5000
max_len = 200

n_epochs = 8
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(tweets_label_test_df["clean_data"].values.astype('U'))
sequences = tokenizer.texts_to_sequences(tweets_label_test_df["clean_data"].values.astype('U'))
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

labels = tf.keras.utils.to_categorical(tweets_label_test_df["label"], dtype="float32")

model = keras.models.load_model('dl/lstm_best_model.h5')

[[   0    0    0 ...    0  153   93]
 [   0    0    0 ...    3    1  171]
 [   0    0    0 ... 1920  875  172]
 ...
 [   0    0    0 ...  236  298   66]
 [   0    0    0 ...  820   10   39]
 [   0    0    0 ... 3975 3976 1033]]


In [21]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    
except:
    strategy = tf.distribute.get_strategy()
    
print('Number of replicas in sync: ', strategy.num_replicas_in_sync)

Number of replicas in sync:  1


In [23]:
#! rm -rf ~/.nv

In [24]:
test_loss, test_acc = model.evaluate(tweets, labels, verbose=2)
print('Model accuracy: ',test_acc)

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv1d/conv1d (defined at <ipython-input-19-3f7bfbfc43d7>:1) ]] [Op:__inference_test_function_1298]

Function call stack:
test_function


In [25]:
bilstm_model = keras.models.load_model('dl/bi_lstm_best_model.h5')

In [26]:
test_loss, test_acc = bilstm_model.evaluate(tweets, labels, verbose=2)
print('Model accuracy: ',test_acc)

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_1/conv1d_1/conv1d (defined at <ipython-input-26-d57d9f2019f0>:1) ]] [Op:__inference_test_function_3723]

Function call stack:
test_function


In [ ]:
gru_model = keras.models.load_model('dl/gru_best_model.h5')

In [ ]:
test_loss, test_acc = gru_model.evaluate(tweets, labels, verbose=2)
print('Model accuracy: ',test_acc)